# Hugging Face Library 'Transformer'およびT5Tokenizerのダウンロード

参考(https://qiita.com/takubb/items/fd972f0ac3dba909c293)これを基に改造し、最新のGoogle Colaboratoryで動作するようにした

In [3]:
!export CUDA_LAUNCH_BLOCKING=1
# !pip install torch
!pip install torchvision
!pip install transformers
#!apt install swig
# Sentencepieceのインストール
!pip install sentencepiece
from transformers import T5Tokenizer
tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-roberta-base")
tokenizer.do_lower_case = True  # due to some bug of tokenizer config loading

In [4]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1" 
import gzip
import shutil
import sqlite3
import random
from math import ceil
import re
import csv
import glob
import torchvision
import statistics
import numpy as np
import lightgbm
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
warnings.filterwarnings('ignore')

# PyTorchとGPU設定

In [5]:
#!pip install torch
import torch
# GPUが使えれば利用する設定
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

# Data Augmentation kansuu

In [6]:
# synreplace - replace kasho kosuu
# randinsert - tasu kotoba no kazu
# randdelete - delete kakuritsu
# randswap - swap kaisuu

class synreplace(object):
    def __init__(self, num):
        self.num = num
        self.model = RobertaForMaskedLM.from_pretrained("rinna/japanese-roberta-base")
    def __call__(self, textlist):
        # textlist: honbun no list
        textlen = torch.where(textlist == 3)[0][0]
        for n in range(self.num):
            # chikan shiro
            masked_idx = random.randint(2, textlen-1)
            textlist[masked_idx] = 6
            # convert to tensor
            token_tensor = torch.tensor(textlist)
            # get the top 10 predictions of the masked token
            self.model = self.model.eval()
            with torch.no_grad():
                outputs = self.model(torch.unsqueeze(token_tensor, 0))
                predictions = outputs[0][0, masked_idx].topk(1)
            for i, index_t in enumerate(predictions.indices):
                index = index_t.item()
            textlist[masked_idx] = index
        return textlist

class randinsert(object):
    def __init__(self, num):
        self.num = num
    def __call__(self, textlist):
        for n in range(self.num):
            insword = textlist[random.randint(1,len(textlist)-1)]
            i = random.randint(1,len(textlist)-1)
#            print('len: ', len(textlist))
#            print(i)
            while textlist[i] == 3:
                i = random.randint(1,len(textlist)-1)
#                print(i)
            textlist = torch.cat([textlist[0:i], torch.tensor([insword]), textlist[i:-1]])
        return textlist

class randdelete(object):
    def __init__(self, num):
        self.num = num
    def __call__(self, textlist):
#        print(textlist.shape)
        for i in range(3,len(textlist)-1):
            if textlist[i] == 3:
                continue
            r = random.uniform(0, 1)
            if r < self.num:
#                textlist.pop(i)
                textlist = torch.cat([textlist[0:i], textlist[i+1:], torch.tensor([3])])
#                print(textlist)
        return textlist

class randswap(object):
    def __init__(self, num):
        self.num = num
    def __call__(self, textlist):
        counter = 0
        #rs_sents = np.zeros(len(textlist), dtype=object)
        for i in range(len(textlist)):
            while self.num > counter:
                box = 0
                random_idx_1 = random.randint(1, len(textlist)-1)
                while textlist[random_idx_1] == 3:
                    random_idx_1 = random.randint(0, len(textlist)-1)
                random_idx_2 = random.randint(1, len(textlist)-1)
                while random_idx_1 == random_idx_2 or textlist[random_idx_2] == 3:
                    random_idx_2 = random.randint(0, len(textlist)-1)
                    # print(random_idx_1, random_idx_2)
                box = textlist[random_idx_1]
                textlist[random_idx_1] = textlist[random_idx_2]
                textlist[random_idx_2] = box
                counter += 1
        return textlist

# Custom Tensor Dataset
https://stackoverflow.com/questions/55588201/pytorch-transforms-on-tensordataset

# データセットの準備

ライブドアニュースコーパスをダウンロード

    ダウンロードしたファイルは圧縮（tar.gz形式）ファイル
    様々なジャンル（IT,スポーツ,家電,映画など）のWEBメディアごとにフォルダに記事がテキストファイルで保存されている
    
以下、ファイルを読み込んで、必要な部分を抽出

In [7]:
#urllib.request.urlretrieve("https://www.rondhuit.com/download/ldcc-20140209.tar.gz", "ldcc-20140209.tar.gz")
# ダウンロードした圧縮ファイルのパスを設定
#tgz_fname = "ldcc-20140209.tar.gz" 
# 2つをニュースメディアのジャンルを選定
mydata = '/export/livedoor' 
#処理をした結果を保存するファイル名 
tsv_fname = "all_text.tsv" 

def remove_brackets(inp):
    output = re.sub(u'[〃-〿]', '',(re.sub('＝|=|×|\(|\)|“|”|（|）|／|\[|\]| |　|…|・|\n|\t|/|＜|＞|@|＠', '', re.sub(u'[ℊ-⿻]', '', inp)))) #210A ~ 2FFF
    return output

"""
def read_url(f):
    url = next(f)
    return url[:-1]

def read_date(f):
    date = next(f)
    date = remove_brackets(date.encode().decode('utf-8'))
    return date[:-1]
"""

def read_title(f):
    next(f)
    next(f)
    title = next(f)
    title = remove_brackets(title.encode().decode('utf-8'))
    return title[:-1]

def read_para(f):
    p = ''
    while True:
        try:
            para = next(f)
            para = remove_brackets(para.encode().decode('utf-8'))
            p += para
        except StopIteration:
            break
    return p [:-1]

In [8]:
directory = ['/export/livedoor/dokujo-tsushin', '/export/livedoor/it-life-hack']
target_genre = ["dokujo-tsushin", "it-life-hack"] 
#directory = ['/export/livedoor/dokujo-tsushin', '/export/livedoor/peachy']
#target_genre = ["dokujo-tsushin", "peachy"] 
zero_fnames = []
one_fnames = []

if os.path.exists(tsv_fname) == True:
    with open(tsv_fname, "r+") as f:
        f.truncate(0)

for i in range(2):
    for filename in os.listdir(directory[i]):
        if "LICENSE.txt" in filename:
            continue
        f = os.path.join(directory[i], filename)
        #if os.path.isfile(f):
        #    print(f)
        if target_genre[0] in f and f.endswith(".txt"):
            with open(tsv_fname, "a") as wf:
                writer = csv.writer(wf, delimiter='\t')
                with open(f) as zf:
                    title = read_title(zf)
                    para = read_para(zf)
                    row = [target_genre[0], '0', title, para]
                    writer.writerow(row)
            continue
        if target_genre[1] in f and f.endswith(".txt"):
            with open(tsv_fname, "a") as wf:
                writer = csv.writer(wf, delimiter='\t')
                with open(f) as zf:
                    title = read_title(zf)
                    para = read_para(zf)
                    row = [target_genre[1], '1', title, para]
                    writer.writerow(row)
            continue

pandasでデータを読み込み

In [9]:
import pandas as pd
# データの読み込み
df = pd.read_csv("all_text.tsv", 
                 delimiter='\t', header=None, names=['media_name', 'label','title','sentence'])
# データの確認
print(f'データサイズ： {df.shape}')
df.sample(10)

データサイズ： (1740, 4)


,media_name,label,title,sentence
137,dokujo-tsushin,0,現役ホステスに聞く、一番人気のホステスとは,女性が同性を見る目は厳しい。その視線を意地悪ととるかお勉強になりますと前向きにとらえるかで人...
1228,it-life-hack,1,Webページを丸ごと取り込むワザ！知っているとお得なパソコン快適術知っ得！虎の,ふだん何気なく使っているスマートフォンやパソコンだが、ちょっとした工夫やヒントで効率よく仕事...
1581,it-life-hack,1,シマンテックより警告！メールアドレスを収集する危険なアプリに要注意,シマンテックのセキュリティブログによると、スパム送信用のメールアドレス収集を目的としたAnd...
143,dokujo-tsushin,0,いま話題のフレンチトースト！きな粉＆黒蜜味に、明太マヨ味,このところ、朝夕の情報番組や雑誌などでも注目されているフレンチトースト。江ノ島のLONCAF...
435,dokujo-tsushin,0,コンビニのレジ前でついつい買ってしまうも,日本中、小さな街にもひとつくらいコンビニがある。いつでも気軽に欲しいものが手に入るコンビニは...
394,dokujo-tsushin,0,満たされると不安!?ちょっと不幸な私が好き症候,好きになるのはダメ男ばかり二股かけられる率が高いなぜか既婚者にハマってしまうetc。会うたび...
548,dokujo-tsushin,0,夏バテよりも深刻な新型秋バテは、1日2回の紅茶でリセット!,あつい日がまだまだ続いていますが、みなさん、節電してますか？もとは私たちの配慮だったはずが、...
860,dokujo-tsushin,0,くさったよめがあらわれた！vol.06専業主婦ってやることいっぱい、暇じゃないんだから!?p...,スーパーは、閉店間際が狙いどき！こんにちは！うえやま洋介犬の漫画くさったよめがあらわれた！の...
698,dokujo-tsushin,0,訊かれて困るクリスマスの過ごし方。貴方の楽しみは,独女にとって、良くも悪くも一年で最も神経が過敏になる季節クリスマスが近づいてきた。彼氏や家族...
1373,it-life-hack,1,サービス終了後はどうなる！電子書籍は永遠に読めるのかデジ,ソニーコンピュータエンターテイメントのポータブルゲーム機、PSPで配信されていたコミックコン...


//文章データをsentences、ラベルデータを labelsに保存、以降この2変数だけを利用

In [10]:
mn = df.media_name.values
labels = df.label.values
titles = df.title.values
sentences = df.sentence.values

In [11]:
print(type(labels))

<class 'numpy.ndarray'>


In [12]:
input_ids = []
attention_masks = []

# 1文づつ処理
for sent in sentences:
    encoded_dict = tokenizer.encode_plus(
                        sent,                      
                        add_special_tokens = True, # Special Tokenの追加
                        max_length = 80,  # I think maximum 文章の長さを固定（Padding/Trancatinating）
                        truncation=True,                
                        pad_to_max_length = True,# PADDINGで埋める
                        return_attention_mask = True,   # Attention maksの作成
                        return_tensors = 'pt',     #  Pytorch tensorsで返す
                   )
    
    # https://qiita.com/YuiKasuga/items/343309257da1798c1b63

    # 単語IDを取得    
    input_ids.append(encoded_dict['input_ids'])

    # Attention　maskの取得
    attention_masks.append(encoded_dict['attention_mask'])

# リストに入ったtensorを縦方向（dim=0）へ結合
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

# tenosor型に変換
labels = torch.tensor(labels)

# 確認
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Original:  両親や親族が高齢なってくると、自分の人生を見つめ直さないといけないことがある。自分が独女の場合はなお更である。今回の相談者ミミさん女性38歳は母親が介護が必要になったことを機に働き方を変えたのだが、以降、仕事や人間関係がうまく築けずに悩み続けている。同時に、現在結婚していないことで将来に大きな不安も抱いている。この先、希望を見出すことは出来るのだろうか？霊感鑑定で定評のある占い師翔生先生に鑑定してもらった。占い師翔生先生nifty電話占いメール占い相談内容母の介護が必要になった頃から今までずっと社会から離れ、自宅にこもっているような状態が続いています。母は5年前に亡くなったのですが、それ以前から現在まで時、バイトなどをしているだけで、仕事が決まっても内容が合わなかったり、内定後、仕事先に行ってみると社内の雰囲気が悪かったりと長くは続かず、短期派遣を時折していました。しかし最近は、派遣の仕事さえも決まらず家にずっといるだけの状態になっています。父も高齢になってきており、一人っ子のため、先の事を考えると不安になります。友人も結婚したり、地方に引越したりと付き合いも無くなってきており、母の知人や派遣先で知り合った知人達とはメールでの連絡はとっていたのですが、最近はそんなメールも中止して欲しいと連絡してしまいました。親しくしていた母の親戚とも葬儀後、距離が出てしまい、父方の親戚とも連絡をとっていないため、先の事を考えると本当に不安です。こんな状況がずっと続いているのですが、今後、運命が変わる時は訪れるのでしょうか？鑑定結果はじめましてミミさん。占い師の翔生と申します。早速鑑定してみましょう。2007年の頃よりずっと、あなたは閉鎖的になってしまったようですね。これまでのあなたの努力は、誰にも真似できないほど頑張っていたに違いありません。お母様は深く感謝しておられます。お母様が今でもあなたに感謝しているのはあなたの努力あっての事とお察しします。その当時のあなたはとても精神的に辛く、肉体的にも疲れていた事でしょう。肉親との別れもあって、精神的に燃え尽きてしまった感触があります。その結果、誰かに意見されても、頑張ってきたという気持ちがあるせいか、周りの意見を遮断してしまいがちです。指図されたくない感情の方が優先されてしまうこれからの方向性にも、運勢上その傾向が

In [13]:
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# データセットクラスの作成
dataset = TensorDataset(input_ids, attention_masks, labels)

# 90%地点のIDを取得
#train_size = int(0.9 * len(dataset))
#val_size = len(dataset) - train_size
train_size = 50
val_size = len(dataset) - train_size

# データセットを分割
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('訓練データ数：{}'.format(train_size))
print('検証データ数:　{} '.format(val_size))

# データローダーの作成
batch_size = 32

# 訓練データローダー
train_dataloader = DataLoader(
            train_dataset,  
            sampler = RandomSampler(train_dataset), # ランダムにデータを取得してバッチ化
            batch_size = batch_size
        )

# 検証データローダー
validation_dataloader = DataLoader(
            val_dataset, 
            sampler = SequentialSampler(val_dataset), # 順番にデータを取得してバッチ化
            batch_size = batch_size
        )

len(validation_dataloader)

訓練データ数：50
検証データ数:　1690 


53

In [1]:
train_dataloader


NameError: name 'train_dataloader' is not defined

In [10]:
wcount = 128

h_sentences = []
t_sentences = []

# wcount moji me kara kesu

for i in enumerate(sentences):
    if len(i[1])>wcount:
        h_sentences.append(sentences[i[0]][:wcount])
    else:
        h_sentences.append(sentences[i[0]])

# ushiro kara wcount moji toru

for i in enumerate(sentences):
    if len(i[1])>wcount:
        t_sentences.append(sentences[i[0]][-wcount:])
    else:
        t_sentences.append(sentences[i[0]])

In [12]:
hsentences = np.array(h_sentences)
tsentences = np.array(t_sentences)

# BERT Tokenizerを用いて単語分割・IDへ変換
## Tokenizerの準備
単語分割とIDへ変換

# テスト実行

In [13]:
# 最大単語数の確認
max_len = []
# 1文づつ処理
for sent in hsentences:
    # Tokenizeで分割
    h_token_words = tokenizer.tokenize(sent)
    # 文章数を取得してリストへ格納
    max_len.append(len(h_token_words))
for sent in tsentences:
    # Tokenizeで分割
    t_token_words = tokenizer.tokenize(sent)
    # 文章数を取得してリストへ格納
    max_len.append(len(t_token_words))
# 最大の値を確認
print('最大単語数: ', max(max_len))
print('上記の最大単語数にSpecial token（[CLS], [SEP]）の+2をした値が最大単語数')

最大単語数:  91
上記の最大単語数にSpecial token（[CLS], [SEP]）の+2をした値が最大単語数


In [14]:
h_input_ids = []
t_input_ids = []
h_attention_masks = []
t_attention_masks = []

# 1文づつ処理
for sent in hsentences:
    hencoded_dict = tokenizer.encode_plus(
                        sent,                      
                        add_special_tokens = True, # Special Tokenの追加
                        max_length = wcount+2,  # I think maximum 文章の長さを固定（Padding/Trancatinating）
                        truncation=True,                
                        pad_to_max_length = True,# PADDINGで埋める
                        return_attention_mask = True,   # Attention maksの作成
                        return_tensors = 'pt',     #  Pytorch tensorsで返す
                   )
    # print(hencoded_dict)

    h_input_ids.append(hencoded_dict['input_ids'])
    h_attention_masks.append(hencoded_dict['attention_mask'])
    
for sent in tsentences:
    tencoded_dict = tokenizer.encode_plus(
                        sent,                      
                        add_special_tokens = True, # Special Tokenの追加
                        max_length = wcount+2,  # I think maximum 文章の長さを固定（Padding/Trancatinating）
                        truncation=True,                
                        pad_to_max_length = True,# PADDINGで埋める
                        return_attention_mask = True,   # Attention maksの作成
                        return_tensors = 'pt',     #  Pytorch tensorsで返す
                   )
    # https://qiita.com/YuiKasuga/items/343309257da1798c1b63

    # 単語IDを取得    
    t_input_ids.append(tencoded_dict['input_ids'])

    # Attention　maskの取得
    t_attention_masks.append(tencoded_dict['attention_mask'])

# リストに入ったtensorを縦方向（dim=0）へ結合
h_input_ids = torch.cat(h_input_ids, dim=0)
t_input_ids = torch.cat(t_input_ids, dim=0)
h_attention_masks = torch.cat(h_attention_masks, dim=0)
t_attention_masks = torch.cat(t_attention_masks, dim=0)

# tenosor型に変換
labels = torch.tensor(labels)

# 確認
print('Original: ', hsentences[0])
print('Original: ', tsentences[0])
print('Token IDs:', h_input_ids[0])
print(type(labels), labels)
print(torch.Tensor.size(h_input_ids))
print(torch.Tensor.size(h_attention_masks))
print(torch.Tensor.size(labels))
print(hsentences.size)

Original:  両親や親族が高齢なってくると、自分の人生を見つめ直さないといけないことがある。自分が独女の場合はなお更である。今回の相談者ミミさん女性38歳は母親が介護が必要になったことを機に働き方を変えたのだが、以降、仕事や人間関係がうまく築けずに悩み続けている。同時に
Original:  霊的存在との対話からあなたのこれから迎える環境人間関係などの状況を見通します。nifty電話占いメール占いでは霊感鑑定はもちろん、タロットから運勢の流れを読み解き、また、夢分析から近未来を予測します。関連サイトnifty電話占いメール占い翔生先生プロフィー
Token IDs: tensor([    9,  3652,    26, 13508,    12, 19860,    57,  9123,    20,     7,
         1393,  4221,  1429, 15133,   442,  4569,    20, 19133,  1272,     8,
         5144,  2596,   612,  2855,  3244, 13426,    27,     8, 16058,  5448,
          147,   318,   318,   774,   577,  1300,   559,    11,  4180,    12,
        12079,  3413,   344,   229,  6741,  4294,   478, 20977, 10338,     7,
          945,     7,  2416,    26, 27138,    12,  8988,  6671,  1068,  5176,
        15710,  9146,     8,  2063,     2,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
       

In [15]:
h_input_ids

tensor([[    9,  3652,    26,  ...,     3,     3,     3],
        [    9,  3911,    10,  ...,     3,     3,     3],
        [    9,  1743, 26243,  ...,     3,     3,     3],
        ...,
        [    9,  1387,  1773,  ...,     3,     3,     3],
        [ 3502, 12393,    26,  ...,     3,     3,     3],
        [  673,   223,  5874,  ...,     3,     3,     3]])

In [16]:
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torchvision import transforms, datasets
from transformers import RobertaForMaskedLM

# データセットクラスの作成
# dataset = TensorDataset(h_input_ids, h_attention_masks, labels)
hdataset = TensorDataset(h_input_ids, h_attention_masks, labels)
tdataset = TensorDataset(t_input_ids, t_attention_masks, labels)
# dataset = CustomTensorDataset(tensors = (input_ids, labels), transform = None)


In [17]:
# 80%地点のIDを取得
num_dataset = len(hdataset)
#train_size = int(0.1 * num_dataset)
#val_size = num_dataset - train_size

train_size = 50
val_size = len(hdataset) - train_size

# データセhttp://localhost:8888/notebooks/bert-zuco/augmentation/BERTclassification-mydata-local-augmentation-aug.ipynb#ットを分割
h_train_dataset, h_val_dataset = random_split(hdataset, [train_size, val_size])
t_train_dataset, t_val_dataset = random_split(tdataset, [train_size, val_size])

print('訓練データ数：{}'.format(train_size))
print('検証データ数:　{} '.format(val_size))

# データローダーの作成
batch_size = 32

data_transform = transforms.Compose([
    synreplace(1),
    randinsert(3),
    randdelete(0.15), 
    randswap(2)
])


class MySubset(torch.utils.data.Dataset):
    def __init__(self, dataset, indices, transform=None):
        self.dataset = dataset
        self.indices = indices
        self.transform = transform

    def __getitem__(self, idx):
        xa, mask, label = self.dataset[self.indices[idx]]
        if self.transform:
            xa = self.transform(xa)
        return xa, mask, label

    def __len__(self):
        return len(self.indices)
    
indices = np.random.choice(num_dataset, num_dataset, replace=False)

h_train_dataset = MySubset(hdataset, indices[:train_size], data_transform)
h_val_dataset = MySubset(hdataset, indices[train_size:])
t_train_dataset = MySubset(tdataset, indices[:train_size], data_transform)
t_val_dataset = MySubset(tdataset, indices[train_size:])



# 訓練データローダー
h_train_dataloader = DataLoader(
            h_train_dataset,  
            sampler = RandomSampler(h_train_dataset), # ランダムにデータを取得してバッチ化
            batch_size = batch_size
            )
t_train_dataloader = DataLoader(
            t_train_dataset,
            sampler = RandomSampler(t_train_dataset), # ランダムにデータを取得してバッチ化
            batch_size = batch_size
            )



# 検証データローダー
h_validation_dataloader = DataLoader(
            h_val_dataset, 
            sampler = SequentialSampler(h_val_dataset), # 順番にデータを取得してバッチ化
            batch_size = batch_size
        )
t_validation_dataloader = DataLoader(
            t_val_dataset, 
            sampler = SequentialSampler(t_val_dataset), # 順番にデータを取得してバッチ化
            batch_size = batch_size
        )


hoge = h_validation_dataloader.__iter__()
hoge.next()

訓練データ数：50
検証データ数:　1690 


[tensor([[    9,  1387,  1773,  ...,     3,     3,     3],
         [    9, 12096,    33,  ...,     3,     3,     3],
         [    9, 14193,    31,  ...,     3,     3,     3],
         ...,
         [    9, 13339,  2874,  ...,     3,     3,     3],
         [   77,  1583,  4473,  ...,     3,     3,     3],
         [    9,  8669,     7,  ...,     3,     3,     3]]),
 tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 tensor([1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
         1, 1, 0, 1, 0, 0, 1, 0])]

In [18]:
h_train_dataloader

In [19]:
with open('LGBMmodel.pickle', mode='rb') as fp:
    Lmodel = pickle.load(fp)
'''
    with open('headmodel.pickle', mode='rb') as fp:
    Hmodel = pickle.load(fp)
with open('tailmodel.pickle', mode='rb') as fp:
    Tmodel = pickle.load(fp)
with open('summarymodel.pickle', mode='rb') as fp:
    Smodel = pickle.load(fp)
'''

"\n    with open('headmodel.pickle', mode='rb') as fp:\n    Hmodel = pickle.load(fp)\nwith open('tailmodel.pickle', mode='rb') as fp:\n    Tmodel = pickle.load(fp)\nwith open('summarymodel.pickle', mode='rb') as fp:\n    Smodel = pickle.load(fp)\n"

In [20]:
# Lmodel.predict(train_dataset)  koreugokannte

In [21]:
from transformers import BertForSequenceClassification,AdamW,BertConfig

# BertForSequenceClassification 学習済みモデルのロード
h_model = BertForSequenceClassification.from_pretrained(
    "cl-tohoku/bert-base-japanese-whole-word-masking", # 日本語Pre trainedモデルの指定
    num_labels = 2, # ラベル数（今回はBinaryなので2、数値を増やせばマルチラベルも対応可）
    output_attentions = False, # アテンションベクトルを出力するか
    output_hidden_states = False, # 隠れ層を出力するか
)

t_model = BertForSequenceClassification.from_pretrained(
    "cl-tohoku/bert-base-japanese-whole-word-masking", # 日本語Pre trainedモデルの指定
    num_labels = 2, # ラベル数（今回はBinaryなので2、数値を増やせばマルチラベルも対応可）
    output_attentions = False, # アテンションベクトルを出力するか
    output_hidden_states = False, # 隠れ層を出力するか
)

# モデルをGPUへ転送
h_model.cuda()
t_model.cuda()

2022-10-04 18:27:48.575143: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-04 18:27:48.801596: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-04 18:27:49.318230: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-10-04 18:27:49.318285: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [22]:
# 最適化手法の設定
optimizer = AdamW(h_model.parameters(), lr=2e-5)
optimizer = AdamW(t_model.parameters(), lr=2e-5)

# 訓練パートの定義
def h_train(h_model):
    h_model.train() # 訓練モードで実行
    h_train_loss = 0
    for batch in h_train_dataloader:# train_dataloaderはword_id, mask, labelを出力する点に注意
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        optimizer.zero_grad()
        outputs = h_model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
        loss = outputs.loss
        loss.backward()
        torch.nn.utils.clip_grad_norm_(h_model.parameters(), 1.0)
        optimizer.step()
        h_train_loss += loss.item()
    return h_train_loss

def t_train(t_model):
    t_model.train() # 訓練モードで実行
    t_train_loss = 0
    for batch in t_train_dataloader:# train_dataloaderはword_id, mask, labelを出力する点に注意
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        optimizer.zero_grad()
        outputs = t_model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
        loss = outputs.loss
        loss.backward()
        torch.nn.utils.clip_grad_norm_(t_model.parameters(), 1.0)
        optimizer.step()
        t_train_loss += loss.item()
    return t_train_loss

# テストパートの定義
def h_validation(h_model):
    h_model.eval()# 訓練モードをオフ
    h_val_loss = 0
    with torch.no_grad(): # 勾配を計算しない
        for batch in h_validation_dataloader:
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            with torch.no_grad():        
                (loss, logits) = h_model(b_input_ids, 
                                    token_type_ids=None, 
                                    attention_mask=b_input_mask,
                                    labels=b_labels)
            h_val_loss += loss.item()
    return h_val_loss

def t_validation(t_model):
    t_model.eval()# 訓練モードをオフ
    t_val_loss = 0
    with torch.no_grad(): # 勾配を計算しない
        for batch in t_validation_dataloader:
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            with torch.no_grad():        
                (loss, logits) = t_model(b_input_ids, 
                                    token_type_ids=None, 
                                    attention_mask=b_input_mask,
                                    labels=b_labels)
            t_val_loss += loss.item()
    return t_val_loss

In [ ]:
# 学習の実行
max_epoch = 50
t_train_loss_ = []
t_test_loss_ = []
h_train_loss_ = []
h_test_loss_ = []

for epoch in range(max_epoch):
    h_train_ = h_train(h_model)
    h_test_ = h_train(h_model)
    h_train_loss_.append(h_train_)
    h_test_loss_.append(h_test_)
    t_train_ = t_train(t_model)
    t_test_ = t_train(t_model)
    t_train_loss_.append(t_train_)
    t_test_loss_.append(t_test_)

In [ ]:
# b_input_mask.size(), b_input_ids.size(), labels.size()
# outputs = self.model(torch.unsqueeze(token_tensor, 0))

In [ ]:
h_model.train() # 訓練モードで実行
t_model.train() # 訓練モードで実行
h_train_loss = 0
t_train_loss = 0
for batch in h_train_dataloader:# train_dataloaderはword_id, mask, labelを出力する点に注意
#    b_input_ids = batch[0].to(device)
#    b_input_mask = batch[1].to(device)
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = torch.t(batch[2]).to(device)
    optimizer.zero_grad()
    outputs = h_model(b_input_ids, 
                         token_type_ids=None, 
                         attention_mask=b_input_mask, 
                         labels=b_labels)
    h_loss = outputs.loss
    h_loss.backward()
    torch.nn.utils.clip_grad_norm_(h_model.parameters(), 1.0)
    optimizer.step()
    h_train_loss += h_loss.item()

for batch in t_train_dataloader:# train_dataloaderはword_id, mask, labelを出力する点に注意
#    b_input_ids = batch[0].to(device)
#    b_input_mask = batch[1].to(device)
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = torch.t(batch[2]).to(device)
    optimizer.zero_grad()
    outputs = t_model(b_input_ids, 
                         token_type_ids=None, 
                         attention_mask=b_input_mask, 
                         labels=b_labels)
    t_loss = outputs.loss
    t_loss.backward()
    torch.nn.utils.clip_grad_norm_(t_model.parameters(), 1.0)
    optimizer.step()
    t_train_loss += t_loss.item()

In [ ]:
import statistics

#correct = 0
h_test_accuracy = []
t_test_accuracy = []

# 検証方法の確認（1バッチ分で計算ロジックに確認）

h_model.eval()# 訓練モードをオフ
t_model.eval()# 訓練モードをオフ

for batch in h_validation_dataloader:
    #print(batch)
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    #print(len(b_labels))
    with torch.no_grad():   
        # 学習済みモデルによる予測結果をpredsで取得     
        h_preds = h_model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        h_test_accuracy.append((torch.argmax(h_preds[0], 1) == b_labels).sum().item() / len(b_labels))

for batch in t_validation_dataloader:
    #print(batch)
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    #print(len(b_labels))
    with torch.no_grad():   
        # 学習済みモデルによる予測結果をpredsで取得     
        t_preds = t_model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        t_test_accuracy.append((torch.argmax(t_preds[0], 1) == b_labels).sum().item() / len(b_labels))

        #print('preds: ', torch.argmax(preds[0], 1))
        #print('b_labels: ', b_labels)
'''
        if preds == b_labels:
            correct += 1
            #test_accuracy += torch.sum(preds == b_labels).item() / len(b_labels)

#test_accuracy = correct/len(b_labels)
print(correct)
'''
print(statistics.mean(h_test_accuracy))
print(statistics.mean(t_test_accuracy))

In [ ]:
h_pred_df = pd.DataFrame(np.argmax(h_preds[0].cpu().numpy(), axis=1), columns=['h_pred_label'])
t_pred_df = pd.DataFrame(np.argmax(t_preds[0].cpu().numpy(), axis=1), columns=['t_pred_label'])
label_df = pd.DataFrame(b_labels.cpu().numpy(), columns=['true_label'])
accuracy_df = pd.concat([h_pred_df, t_pred_df, label_df], axis=1)
accuracy_df

In [ ]:
a = torch.randn(4, 4)
print(a)
print(torch.argmax(a))

In [ ]:
# 予測結果の確認
print(f'出力:{h_preds}')

In [ ]:
'''
# 比較しやすい様にpd.dataframeへ整形
import numpy as np
# pd.dataframeへ変換（GPUに乗っているTensorはgpu->cpu->numpy->dataframeと変換）
logits_df = pd.DataFrame(preds[0].cpu().numpy(), columns=['logit_0', 'logit_1'])
## np.argmaxで大き方の値を取得
pred_df = pd.DataFrame(np.argmax(preds[0].cpu().numpy(), axis=1), columns=['pred_label'])
label_df = pd.DataFrame(b_labels.cpu().numpy(), columns=['true_label'])
accuracy_df = pd.concat([logits_df, pred_df, label_df], axis=1)
accuracy_df.head()
'''

In [ ]:
'''
correct = 0
test_accuracy = 0

p_list = []
l_list = []

p_list = pred_df.values.tolist()
l_list = label_df.values.tolist()

#print(type(pred_df))

for i in range(len(b_labels)):
    if(p_list[i] == l_list[i]):
        correct += 1

test_accuracy = correct/len(b_labels)

print(test_accuracy)
'''

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(len(h_test_loss_)), h_test_loss_, label="Head")
plt.plot(range(len(t_test_loss_)), t_test_loss_, label="Tail")
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Error")
plt.show()  # ラベルがあるときは、きちんとplt.show()を呼び出すこと